<a href="https://colab.research.google.com/github/teamgaon/recommendation_algorithms_know/blob/main/211229_sm_know_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Classification Knowcode using KoBERT Model

전체의 절반의 데이터로 kobert 학습
나머지 절반의 데이터를 kobert에 투입해서 knowcode 예측

해당 데이터를 추가한 데이터프레임으로 knowcode 예측 모델 생성

해당 모델로 test셋의 knowcode 예측

-> test 셋 knowcode 예측하는 코드

In [1]:
!pip install gluonnlp
# !pip install mxnet
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
# !pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
!pip install transformers
!pip install sentencepiece
!pip install kobert-transformers
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-6390vb2p/kobert-tokenizer_e699cc73d506411dbe39de587f17cab4
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-6390vb2p/kobert-tokenizer_e699cc73d506411dbe39de587f17cab4
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-tpq1kvnr
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-tpq1kvnr


In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from kobert_tokenizer import KoBERTTokenizer
from sklearn.model_selection import train_test_split
from google.colab import drive
import matplotlib.pyplot as plt

import pandas as pd
from glob import glob
from tqdm import tqdm
import warnings

In [3]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
train1 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2017.csv', encoding='utf-8')
train2 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2018.csv', encoding='utf-8')
train3 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2019.csv', encoding='utf-8')
train4 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2020.csv', encoding='utf-8')

train3 = train3[train3['idx'] != 19871]

test1 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2017_test.csv', encoding='utf-8')
test2 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2018_test.csv', encoding='utf-8')
test3 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2019_test.csv', encoding='utf-8')
test4 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2020_test.csv', encoding='utf-8')

submission = pd.read_csv('/content/gdrive/MyDrive/know/sample_submission.csv', encoding='utf-8')

# train1_1 = train1[int(len(train1)*0.5):]
# train1 = train1[:int(len(train1)*0.5)]
# train2_1 = train2[int(len(train2)*0.5):]
# train2 = train2[:int(len(train2)*0.5)]
# train3_1 = train3[int(len(train3)*0.5):]
# train3 = train3[:int(len(train3)*0.5)]
# train4_1 = train4[int(len(train4)*0.5):]
# train4 = train4[:int(len(train4)*0.5)]

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (82,95,98,109,111,113,127,128,129,132,136) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (93,94,97,98,99,100,104,105,107,112,132,136,140,142) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (179) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (98,109,111,115,117,128,132) have mixed types.Specify dtype option on import or set 

In [5]:
#bq19_1, bq30, bq31, bq32, bq33, bq34, bq38_1
cols = ['bq19_1', 'bq30', 'bq31', 'bq32', 'bq33', 'bq34', 'bq38_1']

def null_to_no(text : str):
  if text == ' ':
    return '없다'
  else:
    return text

for col in cols:
  test1[col] = test1[col].map(null_to_no)

test1['text'] = test1[cols[0]] + ' ' + test1[cols[1]] + ' ' + test1[cols[2]] + ' ' + test1[cols[3]] + ' ' + test1[cols[4]] + ' ' + test1[cols[5]] + ' ' + test1[cols[6]]

In [6]:
# bq28_1	bq29	bq30	bq31	bq32  bq37_1

cols = ['bq28_1',	'bq29',	'bq30',	'bq31',	'bq32', 'bq37_1']

def null_to_no(text : str):
  if text == ' ':
    return '없다'
  else:
    return text

for col in cols:
  test2[col] = test2[col].map(null_to_no)

test2['text'] = test2[cols[0]] + ' ' + test2[cols[1]] + ' ' + test2[cols[2]] + ' ' + test2[cols[3]] + ' ' + test2[cols[4]] + ' ' + test2[cols[5]]

In [7]:
# bq18_10	bq20_1 bq22	bq23	bq24 bq27_1

cols = ['bq18_10',	'bq20_1',	'bq22',	'bq23',	'bq24', 'bq27_1']

def null_to_no(text : str):
  if text == ' ':
    return '없다'
  else:
    return text

for col in cols:
  test3[col] = test3[col].map(null_to_no)

test3['text'] = test3[cols[0]] + ' ' + test3[cols[1]] + ' ' + test3[cols[2]] + ' ' + test3[cols[3]] + ' ' + test3[cols[4]] + ' ' + test3[cols[5]]

In [8]:
# bq18_10 bq20_1 bq26_1

cols = ['bq18_10',	'bq20_1',	'bq26_1']

def null_to_no(text : str):
  if text == ' ':
    return '없다'
  else:
    return text

for col in cols:
  test4[col] = test4[col].map(null_to_no)

test4['text'] = test4[cols[0]] + ' ' + test4[cols[1]] + ' ' + test4[cols[2]]

In [9]:
df = pd.concat([test1[['idx', 'text']], test2[['idx', 'text']], test3[['idx', 'text']], test4[['idx', 'text']]], axis=0)
df = df.reset_index(drop=True)
df

,idx,text
0,0,다른것으로 대체할 수 없는 업무 없다 컴퓨터 없다 없다 없다 비서학
1,1,"제조업이 줄어들 것으로 생각되기 때문 없다 제품검사시스템,PC,엑셀 없다 없다 없다..."
2,2,"1인 미디어 증가 없다 오디션(편집프로그램), 나홀로(방송진행장비) 없다 광선, 홍..."
3,3,선호직업 아님 없다 컴퓨터 없다 없다 없다 화학
4,4,잡지를 위한 정보나 뉴스는 고갈되지 않기 때문에 꾸준히 유지될 것이다 편집기자 뉴스...
...,...,...
35226,35244,없다 전기는 일상생활에 꼭 필요한 것이라서 관련 인력 일자리는 변화가 없을 것이므로...
35227,35245,온라인 설문 도입으로 테블릿을 통해 온라인으로 설문을 받는 방법과 에디팅 방법에 대...
35228,35246,없다 미래 유망 직종 전기공학
35229,35247,기존 노후화된 기계로 교체하기 위해서 새로운 기계 장비를 도입했음 4차산업혁명으로 ...


In [10]:
def remove_ddaom(text:str):
  return str(text).replace('"', '')

df['text'] = df['text'].map(remove_ddaom)

def remove_enter(text:str):
  return text.replace('\n', ' ')

df['text'] = df['text'].map(remove_enter)

In [11]:
## CPU
# device = torch.device("cpu")

## GPU
device = torch.device("cuda:0")

In [12]:
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [14]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [27]:
## Setting parameters
max_len = 256
batch_size = 8
warmup_ratio = 0.1
num_epochs = 20
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [24]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=585,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [25]:
model = torch.load('/content/gdrive/MyDrive/know/model.pt')

In [28]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(sentence):
  data = [sentence, '0']
  dataset_another = [data]
  logits = 0
  another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
  test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=2)
  
  model.eval()

  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
      token_ids = token_ids.long().to(device)
      segment_ids = segment_ids.long().to(device)

      valid_length= valid_length
      label = label.long().to(device)

      out = model(token_ids, valid_length, segment_ids)

      for i in out:
        logits = i
        logits = logits.detach().cpu().numpy()
        logits = np.argmax(logits)

  return logits

label_answer = []
step = 0
for i in tqdm(df['text']):
  label_answer.append(predict(i))

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


100%|██████████| 35231/35231 [4:21:46<00:00,  2.24it/s]


In [29]:
df['label_answer'] = label_answer

In [30]:
df.head()

,idx,text,label_answer
0,0,다른것으로 대체할 수 없는 업무 없다 컴퓨터 없다 없다 없다 비서학,384
1,1,"제조업이 줄어들 것으로 생각되기 때문 없다 제품검사시스템,PC,엑셀 없다 없다 없다...",363
2,2,"1인 미디어 증가 없다 오디션(편집프로그램), 나홀로(방송진행장비) 없다 광선, 홍...",276
3,3,선호직업 아님 없다 컴퓨터 없다 없다 없다 화학,550
4,4,잡지를 위한 정보나 뉴스는 고갈되지 않기 때문에 꾸준히 유지될 것이다 편집기자 뉴스...,283


In [31]:
df.to_csv('/content/gdrive/MyDrive/know/df_test', index=False, encoding='utf8')

## 할일 :
train 셋의 knowcode를 feature로 랜덤포레스트 모델생성

test 셋에 knowcode를 feature로 주고
랜덤포레스트 모델로 knowcode 예측


--> logits 처리 : 꼭 top1으로 할 필요는 없음